In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)


In [2]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimlab.models.distributed_model import DistributedModel
# import cim.data_profile as cim
import json


['/home/d3m614/gridappsd-cim-profile', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/d3m614/.local/lib/python3.8/site-packages', '/home/d3m614/gridappsd-python', '/home/d3m614/CIMHub/src_python', '/home/d3m614/gridappsd-cim-profile', '/home/d3m614/topology-processor', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [3]:
# # Blazegraph connection for running inside the container
# params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
# bg = BlazegraphConnection(params)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

In [5]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [6]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [7]:
# from gridappsd import GridAPPSD
from topology_processor import DistributedTopology

gapps = GridappsdConnection(feeder_mrid)
Topology = DistributedTopology(gapps, feeder_mrid)
topology_response = Topology.create_switch_areas(feeder_mrid)
# topology_response = get_topology_response(feeder_mrid)
topology_response = json.loads(topology_response)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


/home/d3m614/.local/lib/python3.8/site-packages/ipykernel_launcher.py


In [8]:
# topology_response = json.loads(topology_response)

In [9]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response['feeders'])

In [10]:
switch_area = network.switch_areas[0]

# Get phase, bus info about ACLineSegments
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.Terminal)

line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
switch_area = network.switch_areas[0]

switch_area.get_all_attributes(cim.EnergyConsumer)
switch_area.get_all_attributes(cim.EnergyConsumerPhase)

load_ids = list(switch_area.typed_catalog[cim.EnergyConsumer].keys())

for load in load_ids:
    print('p = ', switch_area.typed_catalog[cim.EnergyConsumer][load].p)
    print('q = ', switch_area.typed_catalog[cim.EnergyConsumer][load].q)
    for load_phs in switch_area.typed_catalog[cim.EnergyConsumer][load].EnergyConsumerPhase:
        print('phases: ', load_phs.phase)

In [ ]:
network.get_all_attributes(cim.TransformerTank)
network.get_all_attributes(cim.TransformerTankEnd)
network.get_all_attributes(cim.TransformerTankInfo)
network.get_all_attributes(cim.TransformerEndInfo)
network.get_all_attributes(cim.ShortCircuitTest)
network.get_all_attributes(cim.NoLoadTest)

In [ ]:
network_area=switch_area
network_area.get_all_attributes(cim.ACLineSegment)
network_area.get_all_attributes(cim.ACLineSegmentPhase)
network_area.get_all_attributes(cim.PerLengthPhaseImpedance)
network_area.get_all_attributes(cim.PhaseImpedanceData)

network_area.get_all_attributes(cim.WireSpacingInfo)
network_area.get_all_attributes(cim.WirePosition)
network_area.get_all_attributes(cim.OverheadWireInfo)
network_area.get_all_attributes(cim.ConcentricNeutralCableInfo)
network_area.get_all_attributes(cim.TapeShieldCableInfo)

network_area.get_all_attributes(cim.Terminal) #bus info

In [ ]:
line_ids = list(network.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', network.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', network.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', network.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in network.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
        if line_phs.WireInfo is not None:
            print('gmr: ', line_phs.WireInfo.gmr)
    if network.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance is not None:
        for impedance in network.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance.PhaseImpedanceData:
            pr int('row:', impedance.row, 'col:', impedance.column, 'r:', impedance.r, 'x:', impedance.x, 'b:', impedance.b)
#     if network.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo is not None:
#         for wire_info in network.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo.WireAssemblyInfo:
#             print('row:', wire_info.)

In [ ]:
print(switch_area.get_attributes_query(cim.ConcentricNeutralCableInfo))

In [ ]:
network.typed_catalog[cim.ACLineSegmentPhase][line_id].AssetInfo

In [ ]:
json.loads(network.__dumps__(cim.OverheadWireInfo))

In [ ]:
secondary_area = network.switch_areas[4].secondary_areas[0]
secondary_area.get_all_attributes(cim.PowerElectronicsConnection)
secondary_area.get_all_attributes(cim.PowerElectronicsConnectionPhase)

pec_ids = list(secondary_area.typed_catalog[cim.PowerElectronicsConnection])

for pec in pec_ids:
    print('p = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].p)
    print('q = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].q)
    for pec_phs in secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].PowerElectronicsConnectionPhases:
        print('phase ', pec_phs.phase, ': ', pec_phs.mRID)

In [ ]:
network_area.typed_catalog[cim.ACLineSegment][line_id]

In [ ]:
import sys
path = sys.argv[0]
if path != '/gridappsd/services/gridappsd-topology-processor':
    print(8890)

In [ ]:
values = ['_0649C1EF-E39F-4B46-AD80-D3C180E4562E']
message = bg.sparql.get_class_type_sparql(feeder_mrid, values)
output = bg.execute(message)
out = output['results']['bindings']
if values and not out:
    print('yes')

In [ ]:
cim.ACLineSegmentPhase()